# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1451271469 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/doria/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1451271469 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to owenzhang1990@gmail.com and will expire on October 31, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-18291 - Server binary: /Users/doria/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451271469.log
[INFO] GraphLab Server Version: 1.7.1


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: Number of coefficients    : 4
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 1.049184     | 4146407.600631     | 258679.804477 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+


Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | 87910.0724924  |
| sqft_living |  None | 315.403440552  |
|   bedrooms  |  None | -65080.2155528 |
|  bathrooms  |  None | 6944.02019265  |
+-------------+-------+----------------+
[4 rows x 3 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [6]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.505878


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [114]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predications = model.predict(data)
    # Then compute the residuals/errors
    residuals = outcome - predications
    # Then square and add them up
    RSS = (residuals * residuals).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [40]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

NameError: name 'example_model' is not defined

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [4]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [5]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [6]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [7]:
test_data['lat_plus_long'].mean()

-74.65333497217306

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [45]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [46]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, 
                                                  validation_set = None)
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features,
                                                  validation_set = None)
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features,
                                                  validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 5
PROGRESS: Number of unpacked features : 5
PROGRESS: Number of coefficients    : 6
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.068740     | 4074878.213096     | 236378.596455 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROG

In [49]:
# Examine/extract each model's coefficients:
print model_1.get("coefficients")
print model_2.get("coefficients")
print model_3.get("coefficients")

+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | -56140675.7444 |
| sqft_living |  None | 310.263325778  |
|   bedrooms  |  None | -59577.1160682 |
|  bathrooms  |  None | 13811.8405418  |
|     lat     |  None | 629865.789485  |
|     long    |  None | -214790.285186 |
+-------------+-------+----------------+
[6 rows x 3 columns]

+----------------+-------+----------------+
|      name      | index |     value      |
+----------------+-------+----------------+
|  (intercept)   |  None | -54410676.1152 |
|  sqft_living   |  None | 304.449298057  |
|    bedrooms    |  None | -116366.043231 |
|   bathrooms    |  None | -77972.3305135 |
|      lat       |  None | 625433.834953  |
|      long      |  None | -203958.60296  |
| bed_bath_rooms |  None | 26961.6249092  |
+----------------+-------+----------------+
[7 rows x 3 columns]

+------------------+-------+----------------+
|       name    

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [24]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model_1 = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model_2 = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model_3 = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print rss_model_1, rss_model_2, rss_model_3

9.71328233544e+14 9.61592067856e+14 9.05276314555e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [25]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1 = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2 = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3 = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print rss_model_1, rss_model_2, rss_model_3

2.26568089093e+14 2.24368799994e+14 2.51829318952e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.


In [8]:
import numpy as np

In [9]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe = data_sframe.select_columns(features)
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray = data_sframe.select_column(output)
    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [10]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [11]:
def feature_derivative(errors, feature):
    derivative = 2 * np.dot(errors, feature)
    return(derivative)

In [12]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        predictions = predict_outcome(feature_matrix, weights)
        # compute the errors as predictions - output:
        errors = predictions - output
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares = gradient_sum_squares + derivative * derivative
            # update the weight based on step size and derivative:
            weights[i] = weights[i] - step_size * derivative
        gradient_magnitude = np.sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [13]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [17]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size,tolerance)


In [131]:
sim

In [132]:
test_simple_weights = regression_gradient_descent(test_simple_feature_matrix, test_output,initial_weights, step_size,tolerance)


In [144]:
simple_predict = np.dot(test_simple_feature_matrix, test_simple_weights)

In [146]:
model_features = ['sqft_living', 'sqft_living15']
my_output= 'price'
(model_feature_matrix, model_output) = get_numpy_data(test_data, model_features, my_output)
model_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [147]:
new_model_weights = regression_gradient_descent(model_feature_matrix, model_output, model_weights, step_size,tolerance)


In [148]:
model_predict = np.dot(model_feature_matrix, new_model_weights)

In [118]:
simple_predict 

array([ 356134.44317093,  784640.86422788,  435069.83652353, ...,
        663418.65300782,  604217.10799338,  240550.4743332 ])

In [119]:
test_output

array([ 310000.,  650000.,  233000., ...,  610685.,  400000.,  402101.])

In [121]:
simple_error = simple_predict - test_output

In [123]:
np.sqrt((simple_error * simple_error).sum())

16595181.45707229

In [124]:
model_error = model_predict - model_output

In [125]:
np.sqrt((model_error * model_error).sum())

16427745.400005188

In [87]:
 np.dot(model_2_matrix[0, 0:7], model_2_weights)

476224.60440095473

In [149]:
model_predict

array([ 376094.1267137 ,  754494.14824362,  377778.93547329, ...,
        687596.82507458,  581967.20574374,  218789.22834502])

In [133]:
test_simple_weights

array([-46999.88120687,    282.34308957])

In [134]:
simple_predict = np.dot(test_simple_feature_matrix, test_simple_weights)

In [145]:
simple_predict

array([ 356774.1395186 ,  785960.50864093,  435834.78646219, ...,
        664545.94369185,  605250.45848416,  241006.76363692])

In [139]:
model_predict

array([ 376094.1267137 ,  754494.14824362,  377778.93547329, ...,
        687596.82507458,  581967.20574374,  218789.22834502])

In [15]:
simple_features = ['sqft_living']
my_output= 'price'
(test_simple_feature_matrix, test_output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [151]:
test_simple_weights = regression_gradient_descent(test_simple_feature_matrix, test_output,initial_weights, step_size,tolerance)

In [20]:
model_features = ['sqft_living', 'sqft_living15']
my_output= 'price'
(model_feature_matrix, model_output) = get_numpy_data(train_data, model_features, my_output)
model_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [21]:
new_model_weights = regression_gradient_descent(model_feature_matrix, model_output, model_weights, step_size,tolerance)


In [18]:
np.dot(test_simple_feature_matrix, simple_weights)

array([ 356134.44317093,  784640.86422788,  435069.83652353, ...,
        663418.65300782,  604217.10799338,  240550.4743332 ])

In [156]:
np.dot(model_feature_matrix, new_model_weights)

array([ 376094.1267137 ,  754494.14824362,  377778.93547329, ...,
        687596.82507458,  581967.20574374,  218789.22834502])

In [157]:
test_simple_weights

array([-46999.87880043,    282.35945337])

In [158]:
test_simple_feature_matrix

array([[  1.00000000e+00,   1.43000000e+03],
       [  1.00000000e+00,   2.95000000e+03],
       [  1.00000000e+00,   1.71000000e+03],
       ..., 
       [  1.00000000e+00,   2.52000000e+03],
       [  1.00000000e+00,   2.31000000e+03],
       [  1.00000000e+00,   1.02000000e+03]])

In [160]:
1.43000000e+03 * 282.35945337 -46999.87880043

356774.13951867

In [22]:
model_features = ['sqft_living', 'sqft_living15']
my_output= 'price'
(test_model_feature_matrix, test_model_output) = get_numpy_data(test_data, model_features, my_output)
model_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [23]:
np.dot(test_model_feature_matrix, new_model_weights)

array([ 366651.41203656,  762662.39786164,  386312.09499712, ...,
        682087.39928241,  585579.27865729,  216559.20396617])